# Lifelog Personalization Gatekeeper – Quickstart

This notebook shows how to configure dataset paths, prepare prediction frames, and run the gatekeeper scorecards.

In [ ]:
from pathlib import Path
import os
import sys

PROJECT_ROOT = Path(__file__).resolve().parents[2]
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")

## Configure cache and dataset roots

Override `GATEKEEPER_DATA_ROOT` and `GATEKEEPER_CACHE_DIR` if your datasets live elsewhere.

In [ ]:
DATA_ROOT = PROJECT_ROOT / "memory/lifelog-personalization/datasets"
CACHE_ROOT = PROJECT_ROOT / "memory/lifelog-personalization/.cache"

os.environ.setdefault("GATEKEEPER_DATA_ROOT", str(DATA_ROOT))
os.environ.setdefault("GATEKEEPER_CACHE_DIR", str(CACHE_ROOT))

print("Data root:", os.environ["GATEKEEPER_DATA_ROOT"])
print("Cache root:", os.environ["GATEKEEPER_CACHE_DIR"])


## Available configs

Configs live under `gatekeeper/configs/`. Feel free to copy them into a run-specific directory before editing.

In [ ]:
config_dir = PROJECT_ROOT / "memory/lifelog-personalization/gatekeeper/configs"
for path in sorted(config_dir.glob("*.yaml")):
    print(path.name)


## Build demo predictions

Below we synthesize a minimal set of predictions to exercise the lifelog evaluation pipeline. Replace this block with real model outputs.

In [ ]:
import pandas as pd

config_path = config_dir / "lifelog.yaml"
print("Using config:", config_path.name)

datasets = {
    "lsc24": ["kis", "adhoc", "qa"],
    "ntcir18_lsat": ["dev", "test"],
    "ntcir18_lqat": ["dev"],
    "imageclef20": ["test"],
}
retrievers = ["vector_only", "graphrag_global", "raptor_hierarchy"]

predictions = {}
for dataset, splits in datasets.items():
    for split in splits:
        for retriever in retrievers:
            key = f"{dataset}:{split}:{retriever}"
            rows = []
            for idx in range(3):
                ranked = [f"doc_{i}" for i in range(idx, idx + 5)]
                relevant = ranked[:2]
                rows.append(
                    {
                        "query_id": f"{dataset}_{split}_{idx}",
                        "ranked_ids": ranked,
                        "relevant_ids": relevant,
                        "timestamp": "2024-01-01T12:00:00",
                        "predicted_timestamp": "2024-01-01T12:00:00",
                        "entities": ["alice", "office"],
                        "predicted_entities": ["alice", "office"],
                    }
                )
            predictions[key] = pd.DataFrame(rows)
print(f"Prepared {len(predictions)} prediction tables")


## Run evaluation

`evaluate_lifelog` returns a list of `EvaluationResult` objects. The helper below renders a Markdown scorecard that you can publish or attach to CI artifacts.

In [ ]:
from memory.lifelog_personalization.gatekeeper.runners.run_lifelog import evaluate_lifelog
from memory.lifelog_personalization.gatekeeper.reports.render_scorecard import render_scorecard

results = evaluate_lifelog(config_path, predictions)
for result in results[:3]:
    print(result)

artifacts_dir = PROJECT_ROOT / "memory/lifelog-personalization/artifacts"
artifacts_dir.mkdir(exist_ok=True)
report_path = artifacts_dir / "lifelog_demo.md"
render_scorecard(results, title="Demo Lifelog Retrieval", output=report_path)
print("Scorecard saved to", report_path)


## Next steps

- Replace the synthetic predictions with outputs from your retrievers.
- Use the other configs (personalization, long-context, TTL, editing) following the same pattern.
- Commit the generated scorecards into your evaluation reports or CI artifacts.